# CrawSPBO

In [1]:
from selenium import webdriver
from selenium.common.exceptions import StaleElementReferenceException

In [18]:
import opencc

In [2]:
import pandas as pd
import re

In [3]:
# 由於網站定期由 js 動態生成, 我們直接使用 selenium
driver = webdriver.Chrome()
driver.get('http://bf.spbo1.com/')

In [4]:
# 觀察標籤, 發現需要的資料被放在三個 table 中, 但是共用一個表頭
# 於是我們一次取出三個 table, 並拿出表頭
tables = [driver.find_elements_by_css_selector(f'{tag} > table')[0] for tag in ['#bflive', '#bflive2', '#bflive3']]
columns = [td.text for td in tables[0].find_elements_by_css_selector('tbody > tr')[0].find_elements_by_css_selector('td')]
print(columns) 

['', '11月06日', 'KO时间', '主场球队', '比分', '客场球队', '半场', '资料', '主', '让球', '客']


In [8]:
# 取出 table 的內容, 由於該網站更新時會清除重建 table 內的物件
# 因此我們需要捕捉 StaleElementReferenceException, 重新獲取 tables
while True:
    try:
#         我試著直接拿到 td 的文字, 但是很遺憾的是雙層迴圈取值速度比不上網頁更新, 
#         data = [ [td.text for td in tr.find_elements_by_css_selector('td')]
#                     for tr in tables[0].find_elements_by_css_selector('tbody > tr')[1:] 
#                     if len(tr.find_elements_by_css_selector('td'))==13]

        # 因此退而求其次, 再 tr 的結點就拿出文字, 等等多花些時間清洗
        data = [ tr.text for tr in tables[0].find_elements_by_css_selector('tbody > tr')[1:] 
                    if len(tr.find_elements_by_css_selector('td'))==13]
        break
    except StaleElementReferenceException:
        tables = [driver.find_elements_by_css_selector(f'{tag} > table')[0] for tag in ['#bflive', '#bflive2', '#bflive3']]    

In [9]:
# 同上 (tables[0])
while True:
    try:
        data1 = [tr.text for tr in tables[1].find_elements_by_css_selector('tbody > tr')
                    if len(tr.find_elements_by_css_selector('td'))==13]
        break
    except StaleElementReferenceException :
        tables = [driver.find_elements_by_css_selector(f'{tag} > table')[0] for tag in ['#bflive', '#bflive2', '#bflive3']]    

In [10]:
# 同上 (tables[0])
while True:
    try:
        data2 = [tr.text for tr in tables[2].find_elements_by_css_selector('tbody > tr')
                    if len(tr.find_elements_by_css_selector('td'))==13]
        break
    except StaleElementReferenceException :
        tables = [driver.find_elements_by_css_selector(f'{tag} > table')[0] for tag in ['#bflive', '#bflive2', '#bflive3']]    

In [11]:
# 首先將三個表合併在一起
data_all = []
for d in [data, data1, data2]:
    data_all.extend(d)

In [12]:
# 由於等等需要用空白分隔, 因此比分的欄位內的空白需要拿掉
for i, row in enumerate(data_all) :
    if re.search('[0-9] - [0-9]', row):
        data_all[i] = row.replace(' - ', '-')

In [13]:
# 用空白分隔每一個 tr 的資料
data_all = [d.split(' ') for d in data_all]

In [14]:
# 清洗拿到的資料
for row in data_all :
    
    # 尚未開始的比賽, 我們需要自己給它一個空值
    if row[2] not in ['上', '中', '下', '完', '']:
        row.insert(2, '')
        
    # 不知道這什麼欄位, 但是這欄位會有空值需要自己塞
    if row[6] == '析':
        row.insert(6, '')
        
    # 最後的三個欄位常常有空值, 同樣自己塞
    if row[-1] == '析':
        row.extend(['']*3)

In [15]:
# 確定一下我們每筆資料欄位數都正確了
assert len(data_all) == len([d for d in data_all if len(d)==11])

In [16]:
# 轉換成 DataFrame
df = pd.DataFrame(data=data_all)
# 最後微調一下資料內容
df[1] = df[1]+df[2]
df.drop(columns=2, inplace=True)
df.columns = columns[1:]

In [30]:
# 順便介紹一下opencc, 這是中文轉換的套件
# 實體化轉換器, 餐數給s2t 是簡轉繁, t2s就是繁轉簡啦
s2t = opencc.OpenCC('s2t')

# 將 columns跟 df都進行轉換
columns = [s2t.convert(col) for col in columns]
df = pd.read_json(
        s2t.convert(
            df.to_json(force_ascii=False)))

# 由於 json格式下沒有順序, 因此我們用一開始的 columns重新排序
df = df[columns[1:]]

# 還有 index也會亂掉, 不要忘記了
df = df.sort_index()

In [31]:
# 打完收工
df

,11月06日,KO時間,主場球隊,比分,客場球隊,半場,資料,主,讓球,客
0,敘甲,90+'下,[13]賈柏萊,2-1,警察俱樂部[4],0-1,析,,,
1,沙地甲,90+'下,[19]卡索瑪,1-2,卡赫利塞哈特[11],1-2,析,,,
2,沙地甲,90+'下,[9]瓦沙姆,0-1,阿爾吉魯[17],0-0,析,0.49,0,1.49
3,沙地甲,78'下,[5]德鹹科,1-0,納積蘭[7],1-0,析,,,
4,以乙北,81'下,[4]提比哩亞,3-0,耶路撒冷夏普爾[10],1-0,析,0.73,0,1.09
5,以丙,82'下,聖達拉吉波,4-2,提拉哈卡默爾夏普爾,2-2,析,0.52,0,1.42
6,埃超,58'下,[16]諾高莫斯塔,0-2,佩特捷德[15],0-2,析,0.95,0,0.93
7,阿蘇盃,62'下,穆桑納赫,0-0,米爾巴特[阿曼超3],0-0,析,,,
8,阿蘇盃,58'下,[阿曼超5]蘇哈爾,2-1,凡亞,1-1,析,,,
9,阿聯U21,54'下,瓦斯爾U21,0-2,阿爾賈茲拉U21,0-1,析,0.95,0,0.87
